[Homework for AI-For-Beginners course: 05 - Frameworks](https://github.com/microsoft/AI-For-Beginners/blob/main/lessons/3-NeuralNetworks/05-Frameworks/lab/LabFrameworks.ipynb)

In [1]:
# display all outputs from Jupyter notebook cells, not just last.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datasets import load_dataset
import torch
import numpy as np

In [2]:
# load data
dataset = load_dataset("mnist")

In [7]:
# convert into expected torch tensors with image tensor being of shape (batch_size, 784)
def normalize(examples):
    if "image" in examples:
        examples["image"] = [torch.from_numpy(np.array(img)).view(-1).float() for img in examples["image"]]
    if "label" in examples:
        examples["label"] = torch.from_numpy(np.array(examples["label"])).long()
    return examples

dataset = dataset.with_transform(normalize)

In [10]:
dataset_train, dataset_test = dataset["train"], dataset["test"]
dataset_test_x = torch.stack(dataset_test['image'])
dataset_test_y = dataset_test['label']

In [11]:
dataloader = torch.utils.data.DataLoader(dataset_train, batch_size=32)

In [4]:
def accuracy(dataset_test_x, dataset_test_y):
  _, predicted = torch.max(net(dataset_test_x), 1)
  acc = (predicted == dataset_test_y).float().mean()
  return acc

def train(net, dataloader, dataset_test_x, dataset_test_y, epochs=10, lr=0.05):
  optim = torch.optim.Adam(net.parameters(),lr=lr)
  for ep in range(epochs):
    for batch in dataloader:
      x = batch["image"]
      y = batch["label"]
      z = net(x)
      loss = torch.nn.functional.cross_entropy(z,y)
      optim.zero_grad()
      loss.backward()
      optim.step()
    
    acc = accuracy(dataset_test_x, dataset_test_y)
    print(f"Epoch {ep}: last batch loss = {loss}, val acc = {acc}")

In [12]:
# 1 layer network
net = torch.nn.Sequential(torch.nn.Linear(28*28, 10))

train(net,dataloader,dataset_test_x,dataset_test_y,epochs=2,lr=0.03)

Epoch 0: last batch loss = 46.259090423583984, val acc = 0.8467000126838684
Epoch 1: last batch loss = 89.85942077636719, val acc = 0.857200026512146


In [6]:
# 2 layer network
net = torch.nn.Sequential(
    torch.nn.Linear(28*28, 100),  # First layer with 100 neurons
    torch.nn.ReLU(),  # Non-linear activation function
    torch.nn.Linear(100, 10)  # Second layer with 10 neurons (one for each class)
)

train(net,dataloader,dataset_test_x,dataset_test_y,epochs=2,lr=0.03)

Epoch 0: last batch loss = 2.298203229904175, val acc = 0.10869999974966049
Epoch 1: last batch loss = 2.3001668453216553, val acc = 0.10300000011920929
